# 00 — Master Orchestrator: Bronze → Silver Processing

Main orchestration notebook for processing parquet files through Bronze and Silver layers.

## Architecture Overview

```
Parquet Files (Files/{source}/{run_ts}/)
    ↓
Bronze Layer (append with run_ts for CDC)
    ↓
Silver Layer (CDC merge: INSERT/UPDATE/DELETE)
    ↓
Watermark Update (incremental tables only)
```

## Process Flow

1. **Load Configuration** (DAG, enabled tables, retry filter)
2. **Check Incremental** → Run watermark merge if needed
3. **Bronze Processing** → Parallel table loading (10 workers)
4. **Bronze Logging** → Batch log all results
5. **Silver Processing** → Parallel CDC merge (tables with business_keys)
6. **Silver Logging** → Batch log all results
7. **Summary Statistics** → Performance metrics, efficiency

## Key Features

- **Parallel Processing**: ThreadPoolExecutor for 5-10x speedup
- **Idempotency**: Check logs before reprocessing
- **Retry Support**: Process only specific tables
- **Error Resilience**: Continue on failure, comprehensive logging
- **Performance Tracking**: Efficiency metrics (theoretical vs actual time)

## Parameters

- `source`: Source system name (e.g., "vizier")
- `run_ts`: Run timestamp (e.g., "20251105T142752505")
- `dag_path`: Path to DAG configuration JSON
- `retry_tables`: Optional list of tables to retry
- `force_reload`: Ignore log and reload all
- `max_workers`: Parallel workers (default: 10)
- `debug`: Enable debug output

In [ ]:
# Parameters (Papermill compatible)
source = "anva_meeus"                               # Source system name
run_ts = "20251001T183103260"                       # Run timestamp
dag_path = "config/dag_anva_meeus_week.json"        # DAG configuration path
retry_tables = None                                 # Optional: list of table names to retry
force_reload = True                                # If True, ignore logs and reload all
max_workers = 10                                    # Parallel workers for table processing
debug = True                                        # Enable debug output

## [1] Setup and Imports

In [ ]:
from datetime import datetime, timezone
from concurrent.futures import ThreadPoolExecutor, as_completed
from typing import List, Dict, Any
import json
from uuid import uuid4

print("="*80)
print("MASTER ORCHESTRATOR STARTING")
print("="*80)
print(f"Source: {source}")
print(f"Run TS: {run_ts}")
print(f"DAG: {dag_path}")
print(f"Retry tables: {retry_tables}")
print(f"Force reload: {force_reload}")
print(f"Max workers: {max_workers}")
print(f"Debug: {debug}")
print("="*80)

## [2] Load Utility Notebooks

# Load logging utilities
# Load config utilities


In [ ]:
# Debug: Check working directory
from pathlib import Path
import os

print(f"Current working directory: {os.getcwd()}")
print(f"Path.cwd(): {Path.cwd()}")
print(f"\nFiles in current dir:")
print(list(Path.cwd().iterdir())[:10])

print(f"\nDoes notebooks/ exist from here? {Path('notebooks').exists()}")
print(f"Does ../notebooks/ exist? {Path('../notebooks').exists()}")
print(f"Does 01_process_data.ipynb exist? {Path('01_process_data.ipynb').exists()}")

In [ ]:
## [1.5] Initialize Spark Session

# Check if Spark session exists (Fabric/Databricks has it by default)
# For local environments, create it
try:
    spark
    print("✓ Spark session already available")
except NameError:
    print("Creating Spark session for local environment...")
    from pyspark.sql import SparkSession
    
    spark = SparkSession.builder \
        .appName("DWH_Bronze_Silver_Processing") \
        .enableHiveSupport() \
        .getOrCreate()
    
    print("✓ Spark session created")

spark.conf.set("spark.sql.parquet.datetimeRebaseModeInRead", "CORRECTED")
spark.conf.set("spark.sql.parquet.int96RebaseModeInRead", "CORRECTED")
spark.conf.set("spark.sql.parquet.datetimeRebaseModeInWrite", "CORRECTED")
spark.conf.set("spark.sql.parquet.int96RebaseModeInWrite", "CORRECTED")

# Verify Spark session
print(f"  Spark version: {spark.version}")
print(f"  Application ID: {spark.sparkContext.applicationId}")

In [ ]:
%run "./notebooks/01_utils_logging.ipynb"

In [ ]:
%run "./notebooks/02_utils_config.ipynb"

## [3] Load DAG Configuration

In [ ]:
# Load and validate DAG
print(f"\n📋 Loading DAG configuration...")
dag = load_dag(dag_path)
print(f"✓ DAG loaded: {dag.get('source')}")

# Get metadata
dag_metadata = get_dag_metadata(dag)
base_files = dag_metadata['base_files']

print(f"  Base files: {base_files}")

# Get tables to process
tables_to_process = get_tables_to_process(
    dag=dag,
    retry_tables=retry_tables,
    only_enabled=True
)

# Ensure schemas exist        
schemas = set()

for t in tables_to_process:
    delta_table = t.get("delta_table")
    delta_schema = t.get("delta_schema")

    if delta_table and "." in delta_table:
        # Vorm: schema.tabel in delta_table
        schema = delta_table.split(".")[0]
    else:
        # Anders: gebruik delta_schema of standaard 'bronze'
        schema = (delta_schema or "bronze")

    schemas.add(schema)

for schema in sorted(schemas):
    spark.sql(f"CREATE SCHEMA IF NOT EXISTS `{schema}`")

print("Schemas ensured:", ", ".join(sorted(schemas)))

print(f"\n📊 Tables to process: {len(tables_to_process)}")

# Show summary
dag_summary = summarize_dag(dag)
print(f"  Total enabled: {dag_summary['enabled_tables']}")
print(f"  Load modes: {dag_summary['load_mode_counts']}")

if not tables_to_process:
    print("\n⚠️  No tables to process. Exiting.")
    raise SystemExit(0)

## [4] Generate Run ID

In [ ]:
# Generate unique run ID
RUN_ID = f"{run_ts}_{uuid4().hex[:8]}"
run_date = build_run_date(run_ts)
print(f"\n🆔 Run ID: {RUN_ID}")

## [5] Check for Incremental Tables (Watermark Merge)

If incremental tables are present, run watermark merge notebook.
This must happen BEFORE Bronze loading starts.

In [ ]:
print(f"\n💧 Checking for incremental tables...")

# Filter incremental tables
incremental_tables = get_tables_by_load_mode(tables_to_process, "incremental")

if len(incremental_tables) > 0:
    print(f"  Found {len(incremental_tables)} incremental tables")
    print(f"  Tables: {[t['name'] for t in incremental_tables[:5]]}")
    
    # Get watermarks path from DAG
    wm_configpath = dag_metadata.get('watermarks_path', 'config/watermarks.json')
    
    # Build watermark folder path (where extraction pipeline writes watermarks)
    wm_folder = f"runtime/{source}/{run_ts}/"
    
    print(f"  Config: {wm_configpath}")
    print(f"  Runtime folder: {wm_folder}")
    
    # Note: In Fabric, this would use mssparkutils.notebook.run()
    # For local testing, we skip watermark merge (not critical for Bronze/Silver testing)
    print(f"\n  ⚠️  Watermark merge would run here (11_bronze_watermark_merge.ipynb)")
    print(f"     Skipping for now - watermarks managed by extraction pipeline")
else:
    print(f"  ◯ No incremental tables - skipping watermark merge")

print("="*80)

## [6] Bronze Processing (Parallel)

Load all tables from parquet to Bronze Delta tables in parallel.

In [ ]:
# Load Bronze worker notebook
%run "./notebooks/10_bronze_load.ipynb"

print("✓ Bronze worker loaded")

In [ ]:
# print("Catalogs:")
# spark.sql("SHOW CATALOGS").show(truncate=False)

# print("\nSchemas (databases) in spark_catalog:")
# spark.sql("SHOW DATABASES").show(truncate=False)

# # print("\nTables per schema:")
# # for row in spark.sql("SHOW DATABASES").collect():
# #     db = row["namespace"] if "namespace" in row else row["databaseName"]
# #     print(f"\n=== {db} ===")
# #     spark.sql(f"SHOW TABLES IN {db}").show(truncate=False)

# for db in ["demo"]:
#     print(f"Dropping schema `{db}` (CASCADE)...")
#     spark.sql(f"DROP DATABASE IF EXISTS `{db}` CASCADE")

# print("\nSchemas na drop:")
# spark.sql("SHOW DATABASES").show(truncate=False)


In [ ]:
print(f"\n🔵 BRONZE: Loading parquet to Delta tables...")
print(f"  Workers: {max_workers}")
print(f"  Tables: {len(tables_to_process)}")

bronze_start = datetime.now(timezone.utc)
bronze_results = []

# Filter tables if not force_reload (check logs)
if not force_reload:
    print(f"\n  📋 Checking logs for already processed tables...")
    
    # Get successfully processed tables from log
    processed_tables = get_successful_tables(run_ts, layer="bronze")
    
    if processed_tables:
        print(f"    Found {len(processed_tables)} already processed tables")
        
        # Filter out already processed
        tables_to_process_bronze = [
            t for t in tables_to_process 
            if t['name'] not in processed_tables
        ]

        print(f"    Remaining: {len(tables_to_process_bronze)} tables")
    else:
        tables_to_process_bronze = tables_to_process
else:
    tables_to_process_bronze = tables_to_process
    print(f"  ⚠️  Force reload enabled - processing all tables")

if not tables_to_process_bronze:
    print(f"\n  ✓ All tables already processed for this run_ts")
else:
    print(f"\n  🚀 Processing {len(tables_to_process_bronze)} tables in parallel...\n")
    
    # Wrapper function for parallel execution
    def process_table_wrapper(table_def):
        """Wrapper to catch exceptions and always return a result."""
        try:
            return process_bronze_table(
                table_def=table_def,
                source_name=source,
                run_ts=run_ts,
                base_files=base_files,
                debug=False  # Disable per-table debug in parallel mode
            )
        except Exception as e:
            # If worker throws unhandled exception, create error result
            return {
                "log_id": f"{source}:{table_def['name']}:{run_ts}:error",
                "run_id": RUN_ID,
                "run_ts": run_ts,
                "source": source,
                "table_name": table_def['name'],
                "load_mode": table_def.get('load_mode'),
                "status": "FAILED",
                "rows_read": None,
                "rows_processed": None,
                "start_time": datetime.now(timezone.utc),
                "end_time": datetime.now(timezone.utc),
                "duration_seconds": 0,
                "error_message": f"Unhandled exception: {str(e)[:500]}",
                "parquet_path": None,
                "delta_table": None,
            }
    
    # Set RUN_ID
    RUN_ID = f"{run_ts}"
    print(f"RUN_ID set to {RUN_ID}")
    
    # Parallel execution
    with ThreadPoolExecutor(max_workers=max_workers) as executor:
        futures = {
            executor.submit(process_table_wrapper, table): table 
            for table in tables_to_process_bronze
        }
        
        completed = 0
        for future in as_completed(futures):
            result = future.result()
            bronze_results.append(result)
            completed += 1
            
            # Progress indicator
            status_icon = "✓" if result['status'] == 'SUCCESS' else "✗" if result['status'] == 'FAILED' else "◯"
            # Kort foutfragment erbij (max 120 chars, 1 regel)
            error_snippet = (result.get("error_message") or "")[:120].replace("\n", " ")
            
            print(
                f"[{completed}/{len(tables_to_process_bronze)}]"
                f"{status_icon} {result['table_name']:<30} {result['status']:<10} "
                f"{(result.get('rows_processed') or 0):>10,} rows {error_snippet}"
                )

bronze_end = datetime.now(timezone.utc)
bronze_duration = float((bronze_end - bronze_start).total_seconds())

print(f"\n✓ Bronze processing completed in {bronze_duration}s")

#sys.exit(0)

## [7] Bronze Logging and Summary

In [ ]:
if bronze_results:
    print(f"\n📊 Logging Bronze results...")
        
    # Calculate summary statistics
    success_count = sum(1 for r in bronze_results if r['status'] == 'SUCCESS')
    failed_count = sum(1 for r in bronze_results if r['status'] == 'FAILED')
    empty_count = sum(1 for r in bronze_results if r['status'] == 'EMPTY')
    skipped_count = sum(1 for r in bronze_results if r['status'] == 'SKIPPED')
    
    total_rows = sum(r.get('rows_processed', 0) or 0 for r in bronze_results)
    
    # Performance metrics
    sum_task_seconds = float(sum(r.get('duration_seconds', 0) or 0 for r in bronze_results))
    theoretical_min_sec = float(sum_task_seconds / max_workers if max_workers > 0 else sum_task_seconds)
    actual_time_sec = bronze_duration # float
    efficiency_pct = float((theoretical_min_sec / actual_time_sec * 100) if actual_time_sec > 0 else 0)
    
    # Failed tables list
    failed_tables = [r['table_name'] for r in bronze_results if r['status'] == 'FAILED']
    
    # Log summary
    bronze_summary = {
    "run_id": RUN_ID,
    "run_date": run_date,
    "run_ts": run_ts,
    "source": source,
    "total_tables": len(bronze_results),
    "tables_success": success_count,
    "tables_empty": empty_count,
    "tables_failed": failed_count,
    "tables_skipped": skipped_count,
    "total_rows": total_rows,
    "workers": max_workers,
    "sum_task_seconds": sum_task_seconds,
    "theoretical_min_sec": theoretical_min_sec,
    "actual_time_sec": actual_time_sec,
    "efficiency_pct": efficiency_pct,
    "run_start": bronze_start,
    "run_end": bronze_end,
    "duration_seconds": bronze_duration,
    "error_message": None,
    "failed_tables": failed_tables,
    }

    run_log_id = log_bronze_summary(summary=bronze_summary)

    # Batch log
    log_bronze_batch(bronze_results=bronze_results, run_log_id=run_log_id)

    
    # Print summary
    print(f"\n  Summary:")
    print(f"    Success: {success_count}")
    print(f"    Failed:  {failed_count}")
    print(f"    Empty:   {empty_count}")
    print(f"    Skipped: {skipped_count}")
    print(f"    Total rows: {total_rows:,}")
    print(f"    Efficiency: {efficiency_pct:.1f}%")
    
    if failed_tables:
        print(f"\n  ⚠️  Failed tables: {failed_tables}")
else:
    print(f"\n  ℹ️  No Bronze results to log")

#sys.exit(0)

In [ ]:
spark.sql("SHOW TABLES IN logs").show(truncate=False)
#spark.table("logs.bronze_processing_log").printSchema()
#spark.table("logs.bronze_run_summary").printSchema()

#spark.sql("drop table if exists logs.silver_run_summary").show()
#spark.sql("select * from logs.bronze_run_summary order by run_end desc limit 5").show(truncate=False)

#spark.sql("drop table if exists logs.bronze_processing_log").show()
#spark.sql("drop table if exists logs.bronze_run_summary").show()

# spark.sql("drop table if exists logs.silver_processing_log").show()
# spark.sql("drop table if exists logs.silver_run_summary").show()
# spark.table("logs.bronze_run_summary") \
#       .orderBy("run_end", "source") \
#       .show(20, truncate=False)

# spark.table("logs.bronze_processing_log") \
#       .orderBy("run_ts", "table_name") \
#       .show(200, truncate=False)


## [8] Silver Processing (Parallel CDC Merge)

Process tables that have business_keys defined for CDC merge.

In [ ]:
# Load Silver worker notebook
%run "./notebooks/20_silver_cdc_merge.ipynb"

print("✓ Silver CDC merge worker loaded")
sys.exit(0)

In [ ]:
print(f"\n🔷 SILVER: CDC merge from Bronze...")

# Filter tables for Silver processing:
# 1. Must have business_keys defined
# 2. Must have been successfully loaded to Bronze

successful_bronze_tables = [r['table_name'] for r in bronze_results if r['status'] == 'SUCCESS']

tables_for_silver = [
    t for t in tables_to_process 
    if t.get('business_keys') and t['name'] in successful_bronze_tables
]

print(f"  Tables with business_keys: {len([t for t in tables_to_process if t.get('business_keys')])}")
print(f"  Successful Bronze loads: {len(successful_bronze_tables)}")
print(f"  Tables to process in Silver: {len(tables_for_silver)}")

silver_results = []

if not tables_for_silver:
    print(f"\n  ℹ️  No tables to process in Silver")
else:
    silver_start = datetime.now(timezone.utc)
    
    print(f"\n  🚀 Processing {len(tables_for_silver)} tables in parallel...\n")
    
    # Wrapper function for parallel execution
    def process_silver_wrapper(table_def):
        """Wrapper to catch exceptions and always return a result."""
        try:
            return process_silver_cdc_merge(
                table_def=table_def,
                source_name=source,
                run_ts=run_ts,
                debug=False
            )
        except Exception as e:
            return {
                "log_id": f"{source}:{table_def['name']}:{run_ts}:silver:error",
                "run_id": RUN_ID,
                "run_ts": run_ts,
                "source": source,
                "table_name": table_def['name'],
                "load_mode": table_def.get('load_mode'),
                "status": "FAILED",
                "rows_inserted": None,
                "rows_updated": None,
                "rows_deleted": None,
                "rows_unchanged": None,
                "total_silver_rows": None,
                "bronze_rows": None,
                "bronze_table": None,
                "silver_table": None,
                "start_time": datetime.now(timezone.utc),
                "end_time": datetime.now(timezone.utc),
                "duration_seconds": 0,
                "error_message": f"Unhandled exception: {str(e)[:500]}",
            }
    
    # Parallel execution
    with ThreadPoolExecutor(max_workers=max_workers) as executor:
        futures = {
            executor.submit(process_silver_wrapper, table): table 
            for table in tables_for_silver
        }
        
        completed = 0
        for future in as_completed(futures):
            result = future.result()
            silver_results.append(result)
            completed += 1
            
            status_icon = "✓" if result['status'] == 'SUCCESS' else "✗"
            deletes = result.get('rows_deleted', 0) or 0
            delete_info = f" ({deletes} deleted)" if deletes > 0 else ""
            print(f"    [{completed}/{len(tables_for_silver)}] {status_icon} {result['table_name']:<30} {result['status']:<10}{delete_info}")
            
    
    silver_end = datetime.now(timezone.utc)
    silver_duration = int((silver_end - silver_start).total_seconds())
    
    print(f"\n✓ Silver processing completed in {silver_duration}s")

## [9] Silver Logging and Summary

In [ ]:
if silver_results:
    print(f"\n📊 Logging Silver results...")
    
    # Batch log
    log_silver_batch(silver_results)
    
    # Calculate summary
    success_count = sum(1 for r in silver_results if r['status'] == 'SUCCESS')
    failed_count = sum(1 for r in silver_results if r['status'] == 'FAILED')
    skipped_count = sum(1 for r in silver_results if r['status'] == 'SKIPPED')
    
    total_inserts = sum(r.get('rows_inserted', 0) or 0 for r in silver_results)
    total_updates = sum(r.get('rows_updated', 0) or 0 for r in silver_results)
    total_deletes = sum(r.get('rows_deleted', 0) or 0 for r in silver_results)
    total_unchanged = sum(r.get('rows_unchanged', 0) or 0 for r in silver_results)
    
    failed_tables = [r['table_name'] for r in silver_results if r['status'] == 'FAILED']
    
    # Log summary
    silver_summary = {
        "run_id": RUN_ID,
        "source": source,
        "run_ts": run_ts,
        "run_start": silver_start,
        "run_end": silver_end,
        "duration_seconds": silver_duration,
        "total_tables": len(silver_results),
        "tables_success": success_count,
        "tables_failed": failed_count,
        "tables_skipped": skipped_count,
        "total_inserts": total_inserts,
        "total_updates": total_updates,
        "total_deletes": total_deletes,
        "total_unchanged": total_unchanged,
        "failed_tables": failed_tables,
    }
    
    log_silver_summary(silver_summary)
    
    # Print summary
    print(f"\n  Summary:")
    print(f"    Success: {success_count}")
    print(f"    Failed:  {failed_count}")
    print(f"    Skipped: {skipped_count}")
    if total_inserts or total_updates or total_deletes:
        print(f"    CDC: +{total_inserts or 0} ~{total_updates or 0} -{total_deletes}")
    
    if failed_tables:
        print(f"\n  ⚠️  Failed tables: {failed_tables}")
else:
    print(f"\n  ℹ️  No Silver results to log")

## [10] Final Summary

In [ ]:
total_end = datetime.now(timezone.utc)
total_duration = int((total_end - bronze_start).total_seconds())

print("\n" + "="*80)
print("ORCHESTRATOR SUMMARY")
print("="*80)
print(f"Run ID: {RUN_ID}")
print(f"Source: {source}")
print(f"Run TS: {run_ts}")
print(f"\nTiming:")
print(f"  Bronze: {bronze_duration}s")
if silver_results:
    print(f"  Silver: {silver_duration}s")
print(f"  Total:  {total_duration}s")

print(f"\nBronze Results:")
if bronze_results:
    bronze_success = sum(1 for r in bronze_results if r['status'] == 'SUCCESS')
    bronze_failed = sum(1 for r in bronze_results if r['status'] == 'FAILED')
    print(f"  ✓ Success: {bronze_success}/{len(bronze_results)}")
    if bronze_failed > 0:
        print(f"  ✗ Failed:  {bronze_failed}")
else:
    print(f"  (No processing)")

print(f"\nSilver Results:")
if silver_results:
    silver_success = sum(1 for r in silver_results if r['status'] == 'SUCCESS')
    silver_failed = sum(1 for r in silver_results if r['status'] == 'FAILED')
    print(f"  ✓ Success: {silver_success}/{len(silver_results)}")
    if silver_failed > 0:
        print(f"  ✗ Failed:  {silver_failed}")
else:
    print(f"  (No processing)")

# Overall status
if bronze_results:
    all_bronze_ok = all(r['status'] in ('SUCCESS', 'EMPTY', 'SKIPPED') for r in bronze_results)
else:
    all_bronze_ok = True

if silver_results:
    all_silver_ok = all(r['status'] in ('SUCCESS', 'SKIPPED') for r in silver_results)
else:
    all_silver_ok = True

overall_status = "SUCCESS" if (all_bronze_ok and all_silver_ok) else "PARTIAL" if bronze_results or silver_results else "NO_WORK"

print(f"\nOverall Status: {overall_status}")
print("="*80)

if overall_status != "SUCCESS":
    print(f"\n⚠️  Some tables failed. Check logs for details.")
    print(f"   Use retry_tables parameter to retry specific tables.")
else:
    print(f"\n✓ All processing completed successfully!")